## 8.1 컨볼루션

### 8.1.1 컨볼루션의 역할
- 이산 컨볼루션(discrete convolution) : 2차원 이미지에 가중치 행렬을 스칼라곱을 수행하는 것으로 정의
    * 가중치 행렬 = 커널
  
[특징]  
- 지역성 : 이동된 커널 * 이미지 스칼라곱 
- 평행이동 불변성 : 이미지 전 영역에 대해 동일한 커널 가중치
- 적은 파라미터 : FC와는 달리 컨볼루션에서의 파라미터 수는 이미지 픽셀 수에 의존하지 않음  
    -> 컨볼루션 커널 크기와 모델에서 얼마나 많은 컨볼루션 필터(출력 채널 수)를 쓰는지에 의존
    
## 8.2 컨볼루션 사용해보기
- nn.Conv1d : 시계열용
- nn.Conv2d : 이미지용
- nn.Conv3d : 용적 데이터나 동영상용

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [4]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [5]:
connected_model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

In [6]:
first_model = nn.Sequential(
                nn.Linear(3072, 512),
                nn.Tanh(),
                nn.Linear(512, 2),
                nn.LogSoftmax(dim=1))

In [7]:
conv = nn.Conv2d(3, 16, kernel_size = 3) # 입력 RGB 3, 출력 피처 16, kernel_size=(3, 3)
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [8]:
conv.weight.shape, conv.bias.shape # [출력, 입력, 커널, 커널]

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [9]:
img, _ = cifar2[0]
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape  # B X C X H X W, 출력의 픽셀 30으로 잘림

(torch.Size([1, 3, 32, 32]), torch.Size([1, 16, 30, 30]))

### 8.2.1 경계 패딩하기
- 패딩 여부 상관없이 weight, bias 크기는 변하지 않음
- 컨볼루션과 이미지 크기 변경 문제를 별도로 분리해 기억해야 하는 것을 하나 줄이는데 도움
- 컨볼루션 구조 자체에 더 신경쓸 수 있다.

In [10]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 1, 32, 32]))

In [11]:
conv.weight.shape, conv.bias.shape

(torch.Size([1, 3, 3, 3]), torch.Size([1]))

### 8.2.2 컨볼루션으로 피처 찾아내기
- CH07에서는 weight, bias 랜덤하게 초기화하고, 역전파를 통해 학습되는 파라미터
- 컨볼루션은 직접 가중치 설정 가능

In [12]:
# bias 0으로 제거해 교란 변수 배제
with torch.no_grad() :
    conv.bias.zero_()
    
# 가중치에 상수값 넣어 출력에서의 각 픽셀이 자신의 이웃 픽셀에 대한 평균 가지게
with torch.no_grad() :
    conv.weight.fill_(1.0 / 9.0) # 3 X 3 이웃

In [13]:
conv.bias, conv.weight

(Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([[[[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]],
 
          [[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]],
 
          [[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]]]], requires_grad=True))

In [14]:
output = conv(img.unsqueeze(0))
# 흐려진 이미지 : 각 출력의 픽셀은 자신의 주변 픽셀에 대한 평균이기에 출력 픽셀에서 이러한 상관관계 반영해 픽셀 간의 변화가 부드러워짐

In [15]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)

with torch.no_grad():
    conv.weight[:] = torch.tensor([[-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0]])
    conv.bias.zero_()
    
# 가로로 인접한 두 영역 사이의 수직 경계 탐색

### 8.2.3 깊이와 풀링으로 한 단계 더 인식하기
- 큰 이미지에서 작은 이미지로 다운샘플링  
 : 컨볼루션을 차례로 층층이 쌓으며 동시에 연속적인 컨볼루션 사이의 이미지 다운샘플링
     * 네 개의 픽셀 평균하기 : 평균 풀링
     * 네 개의 픽셀 중 최댓값 : 맥스 풀링 ; 데이터의 3/4 버림
     * stride 하며 컨볼루션 수행하되, n번째 픽셀만 계산

In [16]:
pool = nn.MaxPool2d(2) # 이미지 절반으로 줄임
output = pool(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

### 8.2.4 우리의 신경망에 적용하기

In [21]:
img.shape

torch.Size([3, 32, 32])

In [17]:
model = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2), # 16 X 16 이미지로 다운샘플링
        nn.Conv2d(16, 8, kernel_size=3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2), # 8 X 8
    
        nn.Linear(8 * 8 * 8, 32),  # FC
        nn.Tanh(),
        nn.Linear(32, 2)) # 이진, 멀티 채널의 2차원 피처

## nn.Sequential() : 각 모듈의 출력을 명시적으로 볼 수 없다.

In [18]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [23]:
model(img.unsqueeze(0))
# 선형 계층의 크기는 8 * 8 * 8 = 512에 의존성을 가진다.
# 모델의 용량을 높이려면 컨볼루션층의 출력 채널 수를 늘려 연결되는 선형 계층도 함께 키워줌
# 런타임 오류

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x8 and 512x32)

8 X 8 이미지를 512 요소를 가진 1차원 벡터로 차원 정보를 변경하면 오류 해결 !  
마지막 nn.MaxPool2d(2)의 출력에 대해 **view**를 호출하면 해결할 수 있다.  
하지만, nn.Sequential()은 각 모듈의 출력을 명시적으로 볼 수 없기에 불가능  => nn.Flatten

## 8.3 nn.Module 서브클래싱하기
* 계층 뒤에 다른 계층을 붙이는 것보다 더 복잡한 일을 수행하는 모델을 만들려면 유연성을 위해 nn.Sequential 대신 nn.Module 사용
* 먼저 forward 함수 정의하여 모듈로 입력을 전달하고, 출력 반환하게 해야 한다.  
: 모듈의 연산을 정의하는 영역
* 표준 torch 연산을 사용하기만 한다면 자동미분 기능이 자동으로 역방향 경로를 만들어 주어 nn.Module에는 backward가 필요 없다.
* super().__ init __()을 무조건 호출하기 !

### 8.3.1 nn.Module로 정의된 우리의 신경망
* 분류 신경망의 목적은 일반적으로 큰 수의 픽셀을 가진 이미지에서 출발해 정보를 압축해가면서 분류 클래스로 만들어 가는 것
* 중간에 나타나는 값의 개수가 점점 줄어드는 모습  
-> 컨볼루션의 채널 수가 점점 줄어들고, 풀링에서 픽셀 수가 줄어들며 선형 계층에서는 입력 차원보다 낮은 수의 차원을 출력한다. 빠르게 정보를 축소하는 패턴은 제한된 깊이의 신경망과 작은 이미지에 대해서는 잘 동작하지만, 신경망이 깊어질수록 감소는 천천히 일어난다.
* 최초의 컨볼루션에는 입력 크기에 대해 출력 크기가 줄어들지 않는다.  
만일 출력 픽셀 하나를 32개 채널을 가진 벡터로 본다면, 27개 채널(3채널 X 3 X 3 커널 크기를 가지는 컨볼루션)를 가지는 선형 변환에 해당하며 어느 정도 증가하는 셈이다. 따라서 첫 계층은 예외적으로 데이터의 전체 차원을 크게 증가시킨다.

In [24]:
class Net(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        
        self.fc1 = nn.Linear(8*8*8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x) :
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8*8*8)  # B X N 벡터로
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

### 8.3.2 파이토치가 파라미터와 서브모듈을 유지하는 방법
* 서브모듈은 list나 dict 인스턴스에 들어 있으면 안된다. 그렇지 않으면 옵티마이저가 서브모듈과 파라미터를 찾지 못한다. (nn.ModuleList, nn.ModuleDict는 있다.)
* 훈련 과정이 실제 예측과 매우 다른 경우, 별도의 predict 함수를 만드는 게 낫다. 이런 메소드의 호출은 모듈 자체보다는 forward 메소드를 호출하는 것과 비슷해, hook 호출을 놓치게 되고 __call__을 호출하지 않기 때문에, JIT는 사용시 모듈 구조를 볼 수 없다.  
- hook : 매 layer마다 print문을 통해 확인하지 않아도 각 층의 activation/gradient 값 확인 가능

In [25]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
print(sum(numel_list), numel_list)

18090 [432, 16, 1152, 8, 16384, 32, 64, 2]


### 8.3.3 함수형 API
* 파라미터가 없는 nn.Tanh, nn.MaxPool2d 같은 서브 모듈은 굳이 등록할 필요가 없다. 이들은 view 호출처럼 forward 함수에서 직접 호출하는 것이 더 쉽다.
* '함수형'이란, 내부 상태가 없다는 의미, 즉 출력값이 전적으로 입력 인자에 의존

In [28]:
import torch.nn.functional as F

class Net(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x) :
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [32]:
model = Net()

model(img.unsqueeze(0)) # torch.Size([1, 3, 32, 32])

tensor([[0.0759, 0.0127]], grad_fn=<AddmmBackward0>)

## 8.4 우리가 만든 컨볼루션 신경망 훈련시키기
* 바깥 루프는 epoch 단위로 돌며, 안쪽 루프는 Dataset에서 배치를 만드는 DataLoader 단위로 돈다.

1) 모델에 입력값을 넣고 (순방향 전달)
2) 손실값을 계산하고 (순방향 전달)
3) 이전 기울기 값을 0으로 리셋하고
4) loss.backward()를 호출하여 모든 파라미터에 대한 손실값의 기울기를 계산한다.(역방향 전달)
5) 이후 옵티마이저를 통해 손실값을 낮추도록 파라미터를 조정한다.

In [33]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader) :
    for epoch in range(1, n_epochs + 1) :
        loss_train = 0.0
        for imgs, labels in train_loader : # DataLoader가 만들어준 배치 안에서 데이터셋 순화
            outputs = model(imgs) # 모델에 배치 넣어줌
            
            loss = loss_fn(outputs, labels) # 최소화하려는 손실값 계산
            
            optimizer.zero_grad() # 마지막에 이전 기울기 값 지움
            loss.backward() # 역전파, 신경망이 학습할 모든 파라미터에 대한 기울기 계산
            optimizer.step() # 모델 업데이트
            
            loss_train += loss.item() # epoch 동안 확인한 손실값 모두 더함. 기울기값 꺼내고자 .item() 사용해 손실값을 파이썬 수로 변환 !
            
    if epoch == 1 or epoch % 10 == 0:
            print('Epoch {}, Training loss {}'.format(
                epoch, loss_train / len(train_loader)))  # 배치 단위의 평균 손실값 구하기 위해 훈련 데이터 로더의 길이로 나눔

In [34]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = Net()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [35]:
training_loop(n_epochs = 100, optimizer = optimizer, model = model,
             loss_fn = loss_fn, train_loader = train_loader)

Epoch 100, Training loss 0.16623300351914327


### 8.4.1 정확도 측정

In [36]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

In [46]:
def validate(model, train_loader, val_loader) :
    for name, loader in [("train", train_loader), ("val", val_loader)] :
        correct = 0
        total = 0
        
        with torch.no_grad() :
            for imgs, labels in loader :
                outputs = model(imgs) # model에 배치 넣어줌
                _, predicted = torch.max(outputs, dim=1) # 가장 높은 값을 가진 인덱스 출력
                total += labels.shape[0] # 배치 크기만큼 증가
                correct += int((predicted==labels).sum()) # 확률값이 가장 높았던 클래스와 레이블의 실측값 비교
        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.93
Accuracy val: 0.89


In [50]:
data_path = r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH08\Data/'
torch.save(model.state_dict(), data_path + 'birds_vs_airplanes.pt')

In [52]:
# 저장한 모델 불러오기
loaded_model = Net()
loaded_model.load_state_dict(torch.load(data_path + 'birds_vs_airplanes.pt'))

<All keys matched successfully>

In [56]:
loaded_model 

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=512, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)